In [ ]:

import numpy as np
import pandas as pd
import wikipediaapi
import random


from transformers import AutoTokenizer
import json
import sys
import time
import datetime
import random
import collections
import os

from pathlib import Path
from IPython.display import display, HTML
from datasets import load_dataset, load_metric
from datasets import Dataset
from datasets import DatasetDict
     

transformers.__version__
     
'4.9.2'

impossible_answer = True
model_checkpoint = "indolem/indobert-base-uncased"
batch_size = 16
     
Runtime Settings

!nvidia-smi
     
Wed Aug 11 09:59:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    14W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
Data Preparation
Load Dataset


with open('BACKEND/dev-v2.0.json') as f:
    dev = json.load(f)

with open('BACKEND/train-v2.0.json') as f:
    train = json.load(f)
     

def format(content):
    hf_data = []
    for data in content["data"]:
        title = data["title"]
        for paragraph in data["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                fill = {
                    "id":  qa["id"],
                    "title": title,
                    "context": context,
                    "question": qa["question"],
                    "answers": {"answer_start": [], "text": []}
                }
                answers = qa["plausible_answers"] if qa["is_impossible"] else qa["answers"]
                for answer in answers:
                    fill["answers"]["answer_start"].append(answer["answer_start"])
                    fill["answers"]["text"].append(answer["text"])

                hf_data.append(fill)

    return hf_data
     

%%time
dev_data = format(dev)
train_train = format(train)
     
# CPU times: user 1.46 s, sys: 12.8 ms, total: 1.47 s
# Wall time: 1.48 s

datasets = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(train_data)),
    'validation': Dataset.from_pandas(pd.DataFrame(dev_data))
})
     
print(datasets)
     
# DatasetDict({
#     train: Dataset({
#         features: ['id', 'title', 'context', 'question', 'answers'],
#         num_rows: 130318
#     })
#     validation: Dataset({
#         features: ['id', 'title', 'context', 'question', 'answers'],
#         num_rows: 11858
#     })
# })

# datasets['train']['id'][0]
     
# '56be85543aeaaa14008c9063'
# Data Analyzation

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame((np.asarray(dataset)[picks]).tolist())

    df['answer_start'] = [i['answer_start'] for i in df['answers']]
    df['answer_text'] = [i['text'] for i in df['answers']]

    del df['answers']

    return df
     

display(HTML(show_random_elements(dev_data).to_html()))
     
id	title	context	question	answer_start	answer_text
0	5ad2685dd7d075001a429276	Oxygen	Terkonsentrasi O2 akan memungkinkan pembakaran untuk melanjutkan cepat dan energik. Pipa baja dan pembuluh penyimpanan digunakan untuk menyimpan dan mengirimkan baik oksigen gas dan cair akan bertindak sebagai bahan bakar; dan karena itu desain dan pembuatan sistem O2 membutuhkan pelatihan khusus untuk memastikan bahwa sumber pengapian diminimalkan. Kebakaran yang menewaskan kru Apollo 1 dalam uji coba peluncuran menyebar begitu cepat karena kapsul itu bertekanan dengan O2 murni Tapi pada sedikit lebih dari tekanan atmosfer, bukannya 1⁄3 tekanan normal yang akan digunakan dalam misi.	Bagaimana pipa baja memungkinkan pembakaran untuk melanjutkan?	[65]	[cepat dan energik]
1	5ad4c99d5b96ef001a10a0a5	Warsaw	Pada tahun 1529, Warsawa untuk pertama kalinya menjadi tempat duduk Jenderal Sejm, permanen dari tahun 1569. Pada tahun 1573, kota itu dinamai Konfederasi Warsawa, secara resmi menetapkan kebebasan beragama di Commonwealthuania Polandia. Karena lokasi pusat antara ibu kota Persemakmuran Kraków dan Vilnius, Warsawa menjadi ibu kota Persemakmuran dan Mahkota Kerajaan Polandia ketika Raja Sigismund III Vasa pindah dari Kraków ke Warsawa pada tahun 1596. Pada tahun-tahun berikutnya kota diperluas menuju pinggiran kota. Beberapa distrik independen didirikan, milik para bangsawan dan bangsawan, yang diperintah oleh hukum mereka sendiri. Tiga kali antara tahun 165558 kota itu dikepung dan tiga kali kota itu dijarah dan dijarah oleh pasukan Swedia, Brandenburgian dan Transylvania.	Mengapa Vilnius menjadi ibu kota Persemakmuran?	[238]	[Karena lokasi pusat]
2	5ad2b72fd7d075001a42a024	Rhine	Pada abad ke - 6, Rhine berada di perbatasan Francia. Pada abad ke-9, ia membentuk bagian dari perbatasan antara Francia Tengah dan Barat, tetapi pada abad ke-10, ia sepenuhnya berada di Imperium Romawi Suci, yang mengalir melalui Swabia, Franconia dan Lower Lorraine. Mulut Rhine, di wilayah Belanda, jatuh ke Belanda Burgundian pada abad ke-15; Belanda tetap menjadi daerah yang suka bertengkar di seluruh perang agama Eropa dan runtuhnya Imperium Romawi Suci, ketika Rhine jatuh ke Imperium Prancis Pertama dan negara - negara kliennya. Alsace di tepi kiri Rhine Atas dijual ke Burgundy oleh Archduke Sigismund dari Austria pada tahun 1469 dan akhirnya jatuh ke Prancis dalam Perang Tiga Puluh Tahun. Banyak istana bersejarah di Rhineland-Palatate membuktikan pentingnya sungai sebagai rute komersial.	Kapan kerajaan Belanda terbentuk?	[330]	[pada abad ke-15]
3	5ad40432604f3c001a3ffdbd	Yuan_dynasty	Setelah memperkuat pemerintahannya di Cina bagian utara, Kublai mengejar kebijakan ekspansionis selaras dengan tradisi imperialisme Mongol dan Cina. Dia memperbarui drive besar-besaran melawan dinasti Song ke selatan. Kublai mengepung Xiangyang antara 1268 dan 1273, rintangan terakhir dalam perjalanannya untuk menangkap lembah Sungai Yangzi yang kaya. Suatu ekspedisi angkatan laut yang tidak berhasil dilakukan terhadap Jepang pada tahun 1274. Kublai merebut Modal Song dari Hangzhou pada tahun 1276, kota tersehat di Cina. Para loyalis yang melarikan diri dari ibu kota dan melantik seorang anak kecil sebagai Kaisar Bing Song. Orang Mongol mengalahkan para loyalis pada pertempuran Yamen pada tahun 1279. Kaisar Song yang terakhir tenggelam, mengakhiri dinasti Song. Penaklukan Song kembali ke utara dan selatan Cina untuk pertama kalinya dalam tiga ratus tahun.	Daerah apa yang Kublai coba tangkap dengan mempertahankan Xiangyang?	[322]	[lembah Sungai Yangzi]
4	57094ca7efce8f15003a7dd7	Sky_(United_Kingdom)	BSkyB awalnya dikenakan biaya langganan tambahan untuk menggunakan Sky+ PVR dengan layanan mereka; melepaskan biaya untuk pelanggan yang paketnya berisi dua atau lebih saluran premium. Hal ini berubah seperti 1 Juli 2007, dan sekarang pelanggan yang memiliki Sky+ dan berlangganan untuk setiap paket langganan BSkyB mendapatkan Sky+ termasuk tanpa biaya tambahan. Pelanggan yang tidak berlangganan saluran BSkyB's masih bisa membayar biaya bulanan untuk memungkinkan fungsi Sky+. Pada bulan Januari 2010 BSkyB menghentikan Sky+ Box, membatasi standar Sky Box untuk meningkatkan hanya dan mulai mengeluarkan Sky+HD Box sebagai standar, sehingga memberikan semua pelanggan baru fungsi Sky+. Pada bulan Februari 2011 BSkyB menghentikan varian non-HD dari kotak Multiroom nya, menawarkan versi yang lebih kecil dari kotak SkyHD tanpa Sky+ fungsionalitas. Pada bulan September 2007, Sky meluncurkan kampanye iklan TV baru menargetkan Sky+ pada wanita. Pada tanggal 31 Maret 2008, Sky memiliki 3.393.000 pengguna Sky+.	Kapan Sky meluncurkan iklan TV target kampanye terhadap wanita?	[862, 862, 963]	[September 2007, September 2007, Maret 2008]
5	5710eca0a58dae1900cd6b3e	Huguenot	Pada tahun 1700, beberapa ratus orang Prancis Huguenot bermigrasi dari Inggris ke koloni Virginia, di mana Kerajaan Inggris telah menjanjikan mereka hibah tanah di Lower Norfolk County. Ketika mereka tiba, kalangan berwenang kolonial menawarkan mereka lahan 20 mil di atas air terjun Sungai James, di desa Monacan yang ditinggalkan yang dikenal sebagai kota Manakin, sekarang di Powhatan County. Beberapa pemukim mendarat di Chesterfield County masa kini. Pada tanggal 12 Mei 1705, Majelis Umum Virginia mengadakan aksi naturalisasi 148 Huguenots masih tinggal di Manakintown. Dari 390 pemukim asli di pemukiman terpencil, banyak yang telah meninggal; lain tinggal di luar kota di peternakan dengan gaya Inggris; dan yang lain pindah ke daerah yang berbeda. Secara bertahap mereka kawin dengan tetangga Inggris mereka. Melalui abad ke - 18 dan ke - 19, keturunan Prancis bermigrasi ke barat ke Piedmont, dan melintasi Pegunungan Appalachian ke Barat dari apa yang menjadi Kentucky, Tennessee, Missouri, dan negara - negara lain. Di kawasan Manakintown, Jembatan Peringatan Huguenot di seberang Sungai James dan Jalan Huguenot dinamai untuk menghormati mereka, demikian pula banyak fitur setempat, termasuk beberapa sekolah, termasuk Sekolah Tinggi Huguenot.	Kapan para pemukim ini bersifat alami sebagai kolonis Inggris?	[469, 476, 469]	[12 Mei 1705, 1705, 12 Mei 1705]
6	57377aac1c4567190057447a	Force	Gagasan "force" menyimpan maknanya dalam mekanika kuantum, meskipun satu sekarang berurusan dengan operator bukan variabel klasik dan meskipun fisika sekarang dijelaskan oleh Schrödinger persamaan bukan persamaan Newton. Hal ini memiliki konsekuensi bahwa hasil pengukuran sekarang kadang-kadang "quantized", i.e. mereka muncul dalam bagian diskrit. Hal ini, tentu saja, sulit untuk membayangkan dalam konteks dari "kekuatan". Akan tetapi, potensi V×x,y,z) atau bidang, yang dari mana kekuatan umumnya dapat diturunkan, diperlakukan mirip dengan variabel posisi klasik, yaitu...	Persamaan apa yang menimbulkan ketegangan dalam fisika sebelum persamaan Schrodinger saat ini?	[203, 203, 203, 203, 203]	[persamaan Newton, persamaan Newton, persamaan Newton., persamaan Newton, persamaan Newton]
7	5ad03d2277cf76001a686ecf	Southern_California	Banyak kesalahan mampu menghasilkan skala 6,7+ gempa bumi, seperti San Andreas Fault, yang dapat menghasilkan skala 8,0 peristiwa. Salah lainnya mencakup San Jacinto Fault, Fault Puente Hills, dan Zona Fault Elsinore. USGS telah merilis prakiraan gempa California yang merupakan model gempa bumi di California.	Apa yang UGS California Preview model?	[285]	[gempa bumi di.]
8	573407d7d058e614000b6813	French_and_Indian_War	Pada bulan September 1760, dan sebelum timbul permusuhan, Gubernur Vaudreuil bernegosiasi dari Montreal sebuah penghargaan dengan Jenderal Amherst. Amherst mengabulkan permintaan Vaudreuil bahwa setiap penduduk Prancis yang memilih untuk tetap tinggal di koloni itu akan diberi kebebasan untuk terus beribadat dalam tradisi Katolik Roma, meneruskan kepemilikan tanah milik mereka, dan hak untuk tetap tidak terganggu di rumah mereka. Inggris menyediakan perawatan medis bagi tentara Prancis yang sakit dan terluka serta pasukan reguler Prancis dikembalikan ke kapal - kapal Inggris dengan persetujuan bahwa mereka tidak boleh melayani lagi dalam perang sekarang.	Di September 1760 siapa yang bernegosiasi untuk menyerah dari Montreal?	[58, 58, 58, 58, 58]	[Gubernur Vaudreuil, Gubernur Vaudreuil, Gubernur Vaudreuil, Gubernur Vaudreuil, Gubernur Vaudreuil]
9	572871bd3acd2414000dfa04	Yuan_dynasty	Setelah kematian Tugh Temür pada tahun 1332 dan setelah kematian Rinchinbal (Emperor Ningzong) pada tahun yang sama, Toghun Temür (Emperor Huizong), yang terakhir dari sembilan penerus Kublai Khan, dipanggil kembali dari Guangxi dan berhasil menduduki takhta. Setelah kematian El Temür, Bayan menjadi pejabat yang kuat seperti El Temür pada awal masa pemerintahannya yang panjang. Seraya Torhun Temür bertambah besar, ia mulai tidak menyetujui aturan otokratik Bayan. Pada tahun 1340, ia bersekutu dengan keponakan Bayan Toqto'a, yang sedang berselisih dengan Bayan, dan dibuang Bayan oleh kudeta. Dengan bubarnya Bayan, Meghogha merebut kekuasaan pengadilan. Administrasi pertamanya jelas menunjukkan semangat baru yang baru. Ia juga memberikan beberapa tanda awal tentang arah baru dan positif dalam pemerintahan pusat. Salah satu proyeknya yang sukses adalah menyelesaikan sejarah resmi Liao, Jin, dan Song Dinasti, yang akhirnya selesai pada tahun 1345. Namun, Heghtopha mengundurkan diri dari jabatannya dengan persetujuan dari Toghun Temür, yang menandai akhir pemerintahan pertamanya, dan ia baru dipanggil kembali pada tahun 1349.	Apa nama gaya Cina yang digunakan Rinchinbal?	[76, 76, 76]	[(Emperor Ningzon, (Emperor Ningzon, (Emperor Ningzon]

train_df = pd.DataFrame(train)

train_df['answer_start'] = [i['answer_start'] for i in train_df['answers']]
train_df['answer_text'] = [i['text'] for i in train_df['answers']]

del train_df['answers']
     

dev_df = pd.DataFrame(dev)

dev_df['answer_start'] = [i['answer_start'] for i in dev_df['answers']]
dev_df['answer_text'] = [i['text'] for i in dev_df['answers']]

del dev_df['answers']
     

figsize = (10,6)
train_df['context'].apply(len).plot.hist(title="Contex length histogram", bins=20, figsize=figsize, grid=True)
     
<matplotlib.axes._subplots.AxesSubplot at 0x7f7794fd0f50>


train_df['question'].apply(len).plot.hist(title="Question length histogram", bins=20, figsize=figsize, grid=True)
     
<matplotlib.axes._subplots.AxesSubplot at 0x7ff2836c6a10>


pd.DataFrame([len(i[0]) for i in train_df['answer_text']]).plot.hist(title="Answer length histogram", bins=20, figsize=figsize, grid=True)
     
<matplotlib.axes._subplots.AxesSubplot at 0x7ff26fa0d2d0>


dev_df['context'].apply(len).plot.hist(title="Contex length histogram", bins=20, figsize=figsize, grid=True)
     
<matplotlib.axes._subplots.AxesSubplot at 0x7ff282344a50>


dev_df['question'].apply(len).plot.hist(title="Question length histogram", bins=20, figsize=figsize, grid=True)
     
<matplotlib.axes._subplots.AxesSubplot at 0x7ff26c586250>


pd.DataFrame([len(i[0]) for i in dev_df['answer_text']]).plot.hist(title="Answer length histogram", bins=20, figsize=figsize, grid=True)
     
<matplotlib.axes._subplots.AxesSubplot at 0x7ff26df3db50>

Text Preprocessing

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
     
Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]
Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]
Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]
Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]
Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
     

tokenizer("What is your name?", "My name is Sylvain.")
     
{'input_ids': [3, 11258, 1688, 9724, 14389, 35, 4, 4470, 14389, 1688, 31710, 1589, 18, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
     

for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]
     

len(tokenizer(example["question"], example["context"])["input_ids"])
     
415

len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])
     
384

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)
     

[len(x) for x in tokenized_example["input_ids"]]
     
[384, 171]

for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))
     
[CLS] beyonce menikah tahun 2008 dengan siapa? [SEP] pada tanggal 4 april 2008, beyonce menikahi jay z. dia membeberkan pernikahan mereka di depan umum dalam video montase di pesta mendengarkan album studio ketiganya, aku... sasha fierce, di sony club manhattan pada 22 oktober 2008. aku... sasha fierce dibebaskan pada tanggal 18 november 2008 di amerika serikat. album ini secara resmi memperkenalkan alter ego beyonce sasha fierce, yang dibuat pada tahun 2003 single " crazy in love ", menjual 482. 000 salinan pada minggu pertamanya, debutnya di atas billboard 200, dan memberikan beyonce album nomor satu ketiga berturut - turut di as. album menampilkan lagu nomor satu " single ladies ( put a ring on it ) " dan top - lima lagu " if i were a boy " dan " halo. " mencapai pencapaiannya menjadi 100 wanita lajang terpanjang dalam karirnya, " halo " sukses di amerika serikat membantu beyonce mencapai lebih dari sepuluh single dalam daftar daripada wanita lain pada tahun 2000 - an. ini juga termasuk sukses " sweet dreams ", dan singles " diva ", " ego ", " broken - heardted girl " dan " video phone. " video musik untuk " single ladies " telah rusak dan ditiru di seluruh dunia, melahirkan " kegilaan tari pertama " dari zaman internet menurut toronto star. video ini memenangkan beberapa penghargaan, termasuk best video pada penghargaan musik eropa mtv 2009, penghargaan mobo skotlandia 2009, dan bet awards 2009. pada penghargaan video musik mtv 2009, video itu dinominasikan untuk sembilan penghargaan, akhirnya memenangkan tiga termasuk video of the year. kegagalannya untuk memenangkan kategori video wanita terbaik, yang pergi ke penyanyi pop amerika taylor swift " kau milik aku ", menyebabkan kanye barat mengganggu upacara dan beyonce berimprovisasi re - presentasi penghargaan swift selama pidato penerimaannya sendiri. pada maret 2009, beyonce memulai [SEP]
[CLS] beyonce menikah tahun 2008 dengan siapa? [SEP] " telah rusak dan ditiru di seluruh dunia, melahirkan " kegilaan tari pertama " dari zaman internet menurut toronto star. video ini memenangkan beberapa penghargaan, termasuk best video pada penghargaan musik eropa mtv 2009, penghargaan mobo skotlandia 2009, dan bet awards 2009. pada penghargaan video musik mtv 2009, video itu dinominasikan untuk sembilan penghargaan, akhirnya memenangkan tiga termasuk video of the year. kegagalannya untuk memenangkan kategori video wanita terbaik, yang pergi ke penyanyi pop amerika taylor swift " kau milik aku ", menyebabkan kanye barat mengganggu upacara dan beyonce berimprovisasi re - presentasi penghargaan swift selama pidato penerimaannya sendiri. pada maret 2009, beyonce memulai i am... tour dunia, tur konser kedua di seluruh dunia, yang terdiri dari 108 pertunjukan, menggusarkan $ 119, 5 juta. [SEP]

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])
     
[(0, 0), (0, 3), (3, 6), (6, 7), (8, 15), (16, 21), (22, 26), (27, 33), (34, 39), (39, 40), (0, 0), (0, 4), (5, 12), (13, 14), (15, 20), (21, 25), (25, 26), (27, 30), (30, 33), (33, 34), (35, 43), (44, 47), (48, 49), (49, 50), (51, 54), (55, 66), (67, 77), (78, 84), (85, 87), (88, 93), (94, 98), (99, 104), (105, 110), (111, 115), (115, 118), (119, 121), (122, 127), (128, 140), (141, 146), (147, 153), (154, 163), (163, 164), (165, 168), (168, 169), (169, 170), (170, 171), (172, 175), (175, 177), (178, 180), (180, 183), (183, 184), (184, 185), (186, 188), (189, 193), (194, 198), (199, 208), (209, 213), (214, 216), (217, 224), (225, 229), (229, 230), (231, 234), (234, 235), (235, 236), (236, 237), (238, 241), (241, 243), (244, 246), (246, 249), (249, 250), (251, 261), (262, 266), (267, 274), (275, 277), (278, 286), (287, 291), (292, 294), (295, 302), (303, 310), (310, 311), (312, 317), (318, 321), (322, 328), (329, 334), (335, 349), (350, 353), (353, 355), (356, 359), (360, 363), (363, 366), (366, 367), (368, 371), (371, 373), (374, 376), (376, 379), (379, 380), (380, 381), (382, 386), (387, 393), (394, 398)]

first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])
     
bey Bey

sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)
     
[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]

answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")
     
21 22

print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])
     
jay z
Jay Z

pad_on_right = tokenizer.padding_side == "right"
     

def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples
     

tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)
     
  0%|          | 0/131 [00:00<?, ?ba/s]
  0%|          | 0/12 [00:00<?, ?ba/s]

tokenized_datasets
     
DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 131226
    })
    validation: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 12071
    })
})
Fine Tuning Model

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
     
Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]
Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

args = TrainingArguments(
    f"drive/MyDrive/KOPSI/test-squad",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01
)
     

from transformers import default_data_collator

data_collator = default_data_collator
     

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
     

trainer.train("./drive/MyDrive/KOPSI/test-squad/checkpoint-16404")
     
Loading model from ./drive/MyDrive/KOPSI/test-squad/checkpoint-16404).
You are resuming training from a checkpoint trained with 4.9.1 of Transformers but your current version is 4.9.2. This is not recommended and could yield to errors or unwanted behaviors.
***** Running training *****
  Num examples = 131226
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24606
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 16404
  Will skip the first 2 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.
0it [00:00, ?it/s]
[24243/24606 2:47:11 < 07:44, 0.78 it/s, Epoch 2.96/3]
Epoch	Training Loss	Validation Loss
[24606/24606 3:00:39, Epoch 3/3]
Epoch	Training Loss	Validation Loss
3	1.069500	1.846025
***** Running Evaluation *****
  Num examples = 12071
  Batch size = 16
Saving model checkpoint to drive/MyDrive/KOPSI/test-squad/checkpoint-24606
Configuration saved in drive/MyDrive/KOPSI/test-squad/checkpoint-24606/config.json
Model weights saved in drive/MyDrive/KOPSI/test-squad/checkpoint-24606/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/KOPSI/test-squad/checkpoint-24606/tokenizer_config.json
Special tokens file saved in drive/MyDrive/KOPSI/test-squad/checkpoint-24606/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


TrainOutput(global_step=24606, training_loss=0.36222411965506063, metrics={'train_runtime': 10841.619, 'train_samples_per_second': 36.312, 'train_steps_per_second': 2.27, 'total_flos': 7.715008844873626e+16, 'train_loss': 0.36222411965506063, 'epoch': 3.0})

trainer.save_model("drive/MyDrive/KOPSI/bert-squad-trained")
     
Evaluation

import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()
     
odict_keys(['loss', 'start_logits', 'end_logits'])

output.start_logits.shape, output.end_logits.shape
     
(torch.Size([16, 384]), torch.Size([16, 384]))

output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)
     
(tensor([ 47,  30,  75,  81, 150,  13,  41,  84, 139, 187, 112,  51,  15,  33,
          31, 113], device='cuda:0'),
 tensor([ 47,  37,  79,  82, 150,  14,  41,  98, 144, 191, 113,  52,  22,  33,
          31, 114], device='cuda:0'))

n_best_size = 20
     

import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )
     

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples
     

validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)
     
  0%|          | 0/12 [00:00<?, ?ba/s]

raw_predictions = trainer.predict(validation_features)
     
The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 12071
  Batch size = 16
[755/755 05:35]

validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
     

max_answer_length = 30
     

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers
     
[{'score': 15.697315, 'text': 'Prancis'},
 {'score': 10.089133, 'text': 'Prancis.'},
 {'score': 9.897605, 'text': 'di Prancis'},
 {'score': 9.845985, 'text': 'sebuah kawasan di Prancis'},
 {'score': 9.531105, 'text': 'Normandia, sebuah kawasan di Prancis'},
 {'score': 9.323152,
  'text': 'Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark, Islandia dan Norwegia'},
 {'score': 8.580118, 'text': 'kawasan di Prancis'},
 {'score': 8.531412,
  'text': 'Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark'},
 {'score': 7.5963774,
  'text': 'Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark, Islandia'},
 {'score': 7.263671, 'text': 'Prancis. Mereka'},
 {'score': 6.988349, 'text': 'Prancis. Mereka diturunkan dari Norse ("Norman'},
 {'score': 6.978618, 'text': 'ke Normandia, sebuah kawasan di Prancis'},
 {'score': 5.066856,
  'text': 'Latin: Normanni) adalah orang - orang yang pada abad ke - 10 dan ke - 11 memberikan nama mereka ke Normandia, sebuah kawasan di Prancis'},
 {'score': 4.792348, 'text': ', sebuah kawasan di Prancis'},
 {'score': 4.289422, 'text': 'di Prancis.'},
 {'score': 4.2378035, 'text': 'sebuah kawasan di Prancis.'},
 {'score': 3.9229228, 'text': 'Normandia, sebuah kawasan di Prancis.'},
 {'score': 3.5234408,
  'text': 'di Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark, Islandia dan Norwegia'},
 {'score': 3.403661, 'text': 'di'},
 {'score': 3.352042, 'text': 'sebuah kawasan di'}]

valid_answers[5]
     
{'score': 9.323152,
 'text': 'Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark, Islandia dan Norwegia'}

datasets["validation"][1231]
     
{'answers': {'answer_start': [252, 252, 252],
  'text': ['6.000 kilometer persegi',
   '6.000 kilometer persegi',
   '6.000 kilometer persegi']},
 'context': 'Lebih dari 26.000 kilometer persegi (10.000 mil persegi) lahan pertanian Victoria ditaburkan untuk biji-bijian, sebagian besar di barat negara bagian. Lebih dari 50% dari daerah ini ditabur untuk gandum, 33% untuk barli dan 7% untuk gandum. Lebih jauh 6.000 kilometer persegi (2.300 mil persegi) ditabur untuk jerami. Pada tahun 200304, para petani Victoria menghasilkan lebih dari 3 juta ton gandum dan 2 juta ton barli. Peternakan Victoria menghasilkan hampir 90% buah pir Australia dan ketiga buah apel. Hal ini juga merupakan pemimpin dalam produksi buah batu. Tanaman sayuran utama mencakup asparagus, brokoli, wortel, kentang, dan tomat. Tahun lalu, 121.200 ton pir dan 270.000 ton tomat diproduksi.',
 'id': '570d2d68fed7b91900d45cbd',
 'question': 'Berapa banyak lahan pertanian Victoria tumbuh jerami?',
 'title': 'Victoria_(Australia)'}

import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)
     

from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not impossible_answer:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions
     

final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
     
Post-processing 11858 example predictions split into 12071 features.
  0%|          | 0/11858 [00:00<?, ?it/s]

def print_answer(id):
    text = [i for i in datasets['validation'] if i['id'] == id][0]
    print(f"Text: {text['context']}")
    print(f"Question: {text['question']}")
    print(f"Answer: {final_predictions[id]}")
     

print_answer('5ad39d53604f3c001a3fe8d1')
     
Text: Normans (Norman: musim hujan; Normands; Latin: Normanni) adalah orang - orang yang pada abad ke - 10 dan ke - 11 memberikan nama mereka ke Normandia, sebuah kawasan di Prancis. Mereka diturunkan dari Norse ("Norman" berasal dari "Norseman") perampok dan bajak laut dari Denmark, Islandia dan Norwegia yang, di bawah pemimpin mereka Rollo, setuju untuk bersumpah setia kepada Raja Charles III dari Francia Barat. Melalui generasi asimilasi dan mencampur dengan penduduk asli Frankis dan Romawi-Gaulis, keturunan mereka secara bertahap akan bergabung dengan Carolingian berbasis Francia. Identitas budaya dan etnis yang berbeda dari orang Norman muncul pada paruh pertama abad ke-10, dan terus berkembang selama abad - abad berikutnya.
Question: Yang memberikan nama mereka ke Normandia di 1000 dan 1100
Answer: Normans

metric = load_metric("squad_v2" if impossible_answer else "squad")
     
Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]
Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)
     
{'HasAns_exact': 51.6191600607185,
 'HasAns_f1': 69.09832265202414,
 'HasAns_total': 11858,
 'best_exact': 51.6191600607185,
 'best_exact_thresh': 0.0,
 'best_f1': 69.09832265202414,
 'best_f1_thresh': 0.0,
 'exact': 51.6191600607185,
 'f1': 69.09832265202414,
 'total': 11858}


     